# Tutorial: Inference with Shape-Shifting Traces

This tutorial introduces more advanced techniques for MCMC inference, techniques which are particularly useful when the number of random choices a model makes varies from execution to execution.

In [ ]:
using Gen
using GenViz

In [ ]:
server = VizServer(8094);

In [ ]:
#############
# dirichlet #
#############
import Distributions
struct Dirichlet <: Distribution{Vector{Float64}} end

"""
    dirichlet(alpha::AbstractVector{T}) where {T<:Real}
Sample a `Vector{Float64}` from the Dirichlet distribution with parameter vector `alpha`.
"""
const dirichlet = Dirichlet()

function Gen.logpdf(::Dirichlet, x::AbstractVector{T}, alpha::AbstractVector{U}) where {T, U}
    Distributions.logpdf(Distributions.Dirichlet(alpha), x)
end

function Gen.logpdf_grad(::Dirichlet, x::AbstractVector{T}, alpha::AbstractVector{U}) where {T, U}
    (Distributions.gradlogpdf(Distributions.Dirichlet(alpha), x), nothing, nothing)
end

function Gen.random(::Dirichlet, alpha::AbstractVector{T}) where {T}
    rand(Distributions.Dirichlet(alpha))
end

(::Dirichlet)(alpha) = random(Dirichlet(), alpha)

Gen.has_output_grad(::Dirichlet) = true
Gen.has_argument_grads(::Dirichlet) = (false,)

#############
# geometric #
#############
import Distributions
struct Geometric <: Distribution{Int} end

"""
    geometric(p::Real)
Sample an `Int` from the Geometric distribution with parameter `p`.
"""
const geometric = Geometric()

function Gen.logpdf(::Geometric, x::Int, p::Real)
    Distributions.logpdf(Distributions.Geometric(p), x)
end

function Gen.logpdf_grad(::Geometric, x::Int, p::Real)
    (Distributions.gradlogpdf(Distributions.Geometric(p), x), nothing, nothing)
end

function Gen.random(::Geometric, p::Real)
    rand(Distributions.Geometric(p))
end

(::Geometric)(p) = random(Geometric(), p)

Gen.has_output_grad(::Geometric) = true
Gen.has_argument_grads(::Geometric) = (false,)

In [ ]:
@gen function piecewise_constant(xs::Vector{Float64})
    n = @addr(poisson(1), :segment_count) + 1
    fracs = @addr(dirichlet([1.0 for i=1:n]), :fractions)
    segments = Vector{Float64}(undef, n)
    for i=1:n
        segments[i] = @addr(normal(0, 1), (:segments, i))
    end
    
    noise = @addr(gamma(1,1), :noise)
    
    min_x, max_x = minimum(xs), maximum(xs)
    for (i, x) in enumerate(xs)
        so_far, j = 0, 0
        while j < n && so_far <= (x - min_x) / (max_x - min_x)
            j += 1
            so_far += fracs[j]
        end
        @addr(normal(segments[j], noise), (:y, i))
    end
end;

In [ ]:
xs_dense = collect(range(-5, stop=5, length=50))
ys_simple = fill(1., length(xs_dense)) .+ randn(length(xs_dense)) * 0.1
ys_complex = [Int(floor(abs(x/3))) % 2 == 0 ? 2 : 0 for x in xs_dense] .+ randn(length(xs_dense)) * 0.1;

In [ ]:
function serialize_trace(tr)
    assmt = get_assmt(tr)
    Dict(
      "values" => [assmt[(:segments, i)] for i=1:(assmt[:segment_count]+1)],
      "fracs"  => assmt[:fractions],
      "n" => assmt[:segment_count]+1,
      "noise" => assmt[:noise],
      "y-coords" => [assmt[(:y, i)] for i=1:length(xs_dense)]
    )
end

In [ ]:
viz = Viz(server, joinpath(@__DIR__, "piecewise-constant-viz/dist"), [xs_dense])
for i=1:10
    putTrace!(viz, "t$(i)", serialize_trace(initialize(piecewise_constant, (xs_dense,))[1]))
end
displayInNotebook(viz)

In [ ]:
function make_constraints(ys)
    constraints = DynamicAssignment()
    for i=1:length(ys)
        constraints[(:y, i)] = ys[i]
    end
    constraints
end

In [ ]:
viz = Viz(server, joinpath(@__DIR__, "piecewise-constant-viz/dist"), [xs_dense])
for i=1:10
    (tr, _) = importance_resampling(piecewise_constant, (xs_dense,), make_constraints(ys_simple), 3000)
    putTrace!(viz, "t$(i)", serialize_trace(tr))
end
displayInNotebook(viz)

In [ ]:
viz = Viz(server, joinpath(@__DIR__, "piecewise-constant-viz/dist"), [xs_dense])
scores = Vector{Float64}(undef, 10)
for i=1:10
    (tr, _) = importance_resampling(piecewise_constant, (xs_dense,), make_constraints(ys_complex), 50000)
    scores[i] = get_score(tr)
    putTrace!(viz, "t$(i)", serialize_trace(tr))
end
displayInNotebook(viz)
println(logsumexp(scores)-log(10))

In [ ]:
import Random

In [ ]:
function simple_update(tr)
    (tr, _) = mh(tr, select(:segment_count, :fractions))
    (tr, _) = mh(tr, select(:fractions))
    (tr, _) = mh(tr, select(:noise))
    for i=1:(get_assmt(tr)[:segment_count]+1)
        (tr, _) = mh(tr, select((:segments, i)))
    end
    tr
end

Random.seed!(3)
viz = Viz(server, joinpath(@__DIR__, "piecewise-constant-viz/dist"), [xs_dense])

sc = displayInNotebook(viz) do
    (tr, _) = initialize(piecewise_constant, (xs_dense,), make_constraints(ys_complex))
    for iter=1:500
        tr = simple_update(tr)
        putTrace!(viz, "t", serialize_trace(tr))
        sleep(0.01)
    end
    return get_score(tr)
end
println(sc)

In [ ]:
for i=1:10
    (tr, _) = initialize(piecewise_constant, (xs_dense,), make_constraints(ys_complex))
    for iter=1:500
        tr = simple_update(tr)
    end
    scores[i]=get_score(tr)
end
println(logsumexp(scores)-log(10))

<hr />

### Exercise: Analyzing the algorithm's behavior
Try running the algorithm with random seed 3, then with random seed 4. In both cases, the initial trace has two segments, one long one and one short one. Why does the chain get stuck in the run with random seed 4, but not the run with random seed 3?

*Your answer here*

<hr />

In [ ]:
@gen function segments_proposal(t, xs, ys)
    min_x, max_x = minimum(xs), maximum(xs)
    x_range = max_x - min_x
    n = @addr(poisson(1), :segment_count) + 1
    fracs = @addr(dirichlet([1.0 for i=1:n]), :fractions)
    segments = Vector{Float64}(undef, n)
    for i=1:n
        min = min_x + x_range * sum(fracs[1:i-1])
        max = min_x + x_range * sum(fracs[1:i])
        relevant_ys = [y for (x,y) in zip(xs,ys) if x >= min && x <= max]
        segments[i] = @addr(normal(sum(relevant_ys)/length(relevant_ys), 0.3), (:segments, i))
    end
end

In [ ]:
function custom_update(tr)
    (tr, _) = mh(tr, segments_proposal, (xs_dense,ys_complex))
    (tr, _) = mh(tr, select(:fractions))
    (tr, _) = mh(tr, select(:noise))
    for i=1:(get_assmt(tr)[:segment_count]+1)
        (tr, _) = mh(tr, select((:segments, i)))
    end
    tr
end

viz = Viz(server, joinpath(@__DIR__, "piecewise-constant-viz/dist"), [xs_dense])

scores = displayInNotebook(viz) do
    for i=1:10
        (tr, _) = initialize(piecewise_constant, (xs_dense,), make_constraints(ys_complex))
        for iter=1:500
            tr = custom_update(tr)
            putTrace!(viz, "t$(i)", serialize_trace(tr))
            sleep(0.001)
        end
        scores[i] = get_score(tr)
    end
    scores
end

println(logsumexp(scores)-log(10))

In [ ]:
@gen function split_or_merge(t)
    choices = get_assmt(t)
    old_n = choices[:segment_count] + 1
    
    split_or_merge = @addr(bernoulli(old_n == 1 ? 1 : 0.3), :split_or_merge)
    if split_or_merge
        # split
        idx = @addr(uniform_discrete(1,old_n), :index)
        @addr(beta(1,1), :split_percentage)
        @addr(normal(choices[(:segments, idx)], 0.1), :new_value_1)
        @addr(normal(choices[(:segments, idx)], 0.1), :new_value_2)
    else
        # merge
        idx = @addr(uniform_discrete(1, old_n-1), :index) # merge i and i+1
        @addr(normal((choices[(:segments, idx)] + choices[(:segments, idx+1)]) / 2.0, 0.1), :new_value)
    end
end

function involution(t, fwd_assmt, fwd_ret, proposal_args)
    split_or_merge = fwd_assmt[:split_or_merge]
    old_choices = get_assmt(t)
    n = old_choices[:segment_count] + (split_or_merge ? 2 : 0)
    new_choices = DynamicAssignment()
    bwd_assmt = DynamicAssignment()
    old_fracs = old_choices[:fractions]
    fracs = Vector{Float64}(undef, n)
    idx = fwd_assmt[:index]
    for i=1:idx-1
        fracs[i] = old_fracs[i]
    end
    
    if split_or_merge
        # split
        new_choices[:segment_count] = old_choices[:segment_count] + 1
        proportion = fwd_assmt[:split_percentage]
        fracs[idx] = proportion * old_fracs[idx]
        fracs[idx+1] = (1 - proportion) * old_fracs[idx]
        new_choices[(:segments, idx)] = fwd_assmt[:new_value_1]
        new_choices[(:segments, idx+1)] = fwd_assmt[:new_value_2]
        bwd_assmt[:new_value] = old_choices[(:segments, idx)]
        
        for i=idx+2:n
            fracs[i] = old_fracs[i-1]
            new_choices[(:segments, i)] = old_choices[(:segments, i-1)]
        end
        
        new_choices[:fractions] = fracs
        bwd_assmt[:split_or_merge] = false
        bwd_assmt[:index] = idx
    else
        #merge
        new_choices[:segment_count] = old_choices[:segment_count] - 1
        fracs[idx] = old_fracs[idx] + old_fracs[idx+1]
        new_choices[(:segments, idx)] =  fwd_assmt[:new_value]
        for i=idx+1:n
            fracs[i] = old_fracs[i+1]
            new_choices[(:segments, i)] = old_choices[(:segments, i+1)]
        end
        new_choices[:fractions] = fracs
        bwd_assmt[:split_or_merge] = true
        bwd_assmt[:index] = idx
        bwd_assmt[:new_value_1] = old_choices[(:segments, idx)]
        bwd_assmt[:new_value_2] = old_choices[(:segments, idx+1)]
        bwd_assmt[:split_percentage] = old_fracs[idx] / fracs[idx]
    end
    (new_trace, weight) = force_update(get_args(t), noargdiff, t, new_choices)
    (new_trace, bwd_assmt, weight)
end

In [ ]:
@gen function mean_segments_proposal(t, xs, ys, i)
    min_x, max_x = minimum(xs), maximum(xs)
    x_range = max_x - min_x
    fracs = get_assmt(t)[:fractions]
    min = min_x + x_range * sum(fracs[1:i-1])
    max = min_x + x_range * sum(fracs[1:i])
    relevant_ys = [y for (x,y) in zip(xs,ys) if x >= min && x <= max]
    @addr(normal(sum(relevant_ys)/length(relevant_ys), 0.3), (:segments, i))
end

function custom_update_inv(tr)
    (tr, _) = mh(tr, split_or_merge, (), involution)
    for i=1:(get_assmt(tr)[:segment_count]+1)
        (tr, _) = mh(tr, mean_segments_proposal, (xs_dense, ys_complex, i))
    end
    (tr, _) = mh(tr, select(:noise))
    tr
end

viz = Viz(server, joinpath(@__DIR__, "piecewise-constant-viz/dist"), [xs_dense])

scores = displayInNotebook(viz) do
    for i=1:10
        (tr, _) = initialize(piecewise_constant, (xs_dense,), make_constraints(ys_complex))
        for iter=1:3000
            tr = custom_update_inv(tr)
            putTrace!(viz, "t$(i)", serialize_trace(tr))
        end
        scores[i] = get_score(tr)
    end
    scores
end

println(logsumexp(scores)-log(10))

# Gaussian processes

In [ ]:
import LinearAlgebra

"""Node in a tree representing a covariance function"""
abstract type Node end
abstract type LeafNode <: Node end
abstract type BinaryOpNode <: Node end

"""
    size(::Node)
Number of nodes in the subtree rooted at this node.
"""
Base.size(::LeafNode) = 1
Base.size(node::BinaryOpNode) = node.size

"""Constant kernel"""
struct Constant <: LeafNode
    param::Float64
end


eval_cov(node::Constant, x1, x2) = node.param

function eval_cov_mat(node::Constant, xs::Vector{Float64})
    n = length(xs)
    fill(node.param, (n, n))
end


"""Linear kernel"""
struct Linear <: LeafNode
    param::Float64
end


eval_cov(node::Linear, x1, x2) = (x1 - node.param) * (x2 - node.param)

function eval_cov_mat(node::Linear, xs::Vector{Float64})
    xs_minus_param = xs .- node.param
    xs_minus_param * xs_minus_param'
end


"""Squared exponential kernel"""
struct SquaredExponential <: LeafNode
    length_scale::Float64
end


eval_cov(node::SquaredExponential, x1, x2) =
    exp(-0.5 * (x1 - x2) * (x1 - x2) / node.length_scale)

function eval_cov_mat(node::SquaredExponential, xs::Vector{Float64})
    diff = xs .- xs'
    exp.(-0.5 .* diff .* diff ./ node.length_scale)
end


"""Periodic kernel"""
struct Periodic <: LeafNode
    scale::Float64
    period::Float64
end


function eval_cov(node::Periodic, x1, x2)
    freq = 2 * pi / node.period
    exp((-1/node.scale) * (sin(freq * abs(x1 - x2)))^2)
end


function eval_cov_mat(node::Periodic, xs::Vector{Float64})
    freq = 2 * pi / node.period
    abs_diff = abs.(xs .- xs')
    exp.((-1/node.scale) .* (sin.(freq .* abs_diff)).^2)
end


"""Plus node"""
struct Plus <: BinaryOpNode
    left::Node
    right::Node
    size::Int
end


Plus(left, right) = Plus(left, right, size(left) + size(right) + 1)


function eval_cov(node::Plus, x1, x2)
    eval_cov(node.left, x1, x2) + eval_cov(node.right, x1, x2)
end


function eval_cov_mat(node::Plus, xs::Vector{Float64})
    eval_cov_mat(node.left, xs) .+ eval_cov_mat(node.right, xs)
end


"""Times node"""
struct Times <: BinaryOpNode
    left::Node
    right::Node
    size::Int
end


Times(left, right) = Times(left, right, size(left) + size(right) + 1)


function eval_cov(node::Times, x1, x2)
    eval_cov(node.left, x1, x2) * eval_cov(node.right, x1, x2)
end


function eval_cov_mat(node::Times, xs::Vector{Float64})
    eval_cov_mat(node.left, xs) .* eval_cov_mat(node.right, xs)
end


const CONSTANT = 1      # 0.2
const LINEAR = 2        # 0.2
const SQUARED_EXP = 3   # 0.2
const PERIODIC = 4      # 0.2
const PLUS = 5          # binary 0.1
const TIMES = 6         # binary 0.1

const node_type_to_num_children = Dict(
    CONSTANT => 0,
    LINEAR => 0,
    SQUARED_EXP => 0,
    PERIODIC => 0,
    PLUS => 2,
    TIMES => 2)

const max_branch = 2
const node_dist = Float64[0.2, 0.2, 0.2, 0.2, 0.1, 0.1]

"""Compute covariance matrix by evaluating function on each pair of inputs."""
function compute_cov_matrix(covariance_fn::Node, noise, xs)
    n = length(xs)
    cov_matrix = Matrix{Float64}(undef, n, n)
    for i=1:n
        for j=1:n
            cov_matrix[i, j] = eval_cov(covariance_fn, xs[i], xs[j])
        end
        cov_matrix[i, i] += noise
    end
    return cov_matrix
end


"""Compute covariance function by recursively computing covariance matrices."""
function compute_cov_matrix_vectorized(covariance_fn, noise, xs)
    n = length(xs)
    eval_cov_mat(covariance_fn, xs) + Matrix(noise * LinearAlgebra.I, n, n)
end

function compute_predictive(covariance_fn::Node, noise::Float64,
                            xs::Vector{Float64}, ys::Vector{Float64},
                            new_xs::Vector{Float64})
    n_prev = length(xs)
    n_new = length(new_xs)
    means = zeros(n_prev + n_new)
    cov_matrix = compute_cov_matrix(covariance_fn, noise, vcat(xs, new_xs))
    cov_matrix_11 = cov_matrix[1:n_prev, 1:n_prev]
    cov_matrix_22 = cov_matrix[n_prev+1:n_prev+n_new, n_prev+1:n_prev+n_new]
    cov_matrix_12 = cov_matrix[1:n_prev, n_prev+1:n_prev+n_new]
    cov_matrix_21 = cov_matrix[n_prev+1:n_prev+n_new, 1:n_prev]
    @assert cov_matrix_12 == cov_matrix_21'
    mu1 = means[1:n_prev]
    mu2 = means[n_prev+1:n_prev+n_new]
    conditional_mu = mu2 + cov_matrix_21 * (cov_matrix_11 \ (ys - mu1))
    conditional_cov_matrix = cov_matrix_22 - cov_matrix_21 * (cov_matrix_11 \ cov_matrix_12)
    conditional_cov_matrix = 0.5 * conditional_cov_matrix + 0.5 * conditional_cov_matrix'
    (conditional_mu, conditional_cov_matrix)
end

"""
Predict output values for some new input values
"""
function predict_ys(covariance_fn::Node, noise::Float64,
                    xs::Vector{Float64}, ys::Vector{Float64},
                    new_xs::Vector{Float64})
    (conditional_mu, conditional_cov_matrix) = compute_predictive(
        covariance_fn, noise, xs, ys, new_xs)
    mvnormal(conditional_mu, conditional_cov_matrix)
end

# Model

In [ ]:
@gen function covariance_prior()
    node_type = @addr(categorical(node_dist), :type)

    if node_type == CONSTANT
        param = @addr(uniform_continuous(0, 1), :param)
        node = Constant(param)

    # linear kernel
    elseif node_type == LINEAR
        param = @addr(uniform_continuous(0, 1), :param)
        node = Linear(param)

    # squared exponential kernel
    elseif node_type == SQUARED_EXP
        length_scale= @addr(uniform_continuous(0, 1), :length_scale)
        node = SquaredExponential(length_scale)

    # periodic kernel
    elseif node_type == PERIODIC
        scale = @addr(uniform_continuous(0, 1), :scale)
        period = @addr(uniform_continuous(0, 1), :period)
        node = Periodic(scale, period)

    # plus combinator
    elseif node_type == PLUS
        left = @addr(covariance_prior(), :left)
        right = @addr(covariance_prior(), :right)
        node = Plus(left, right)

    # times combinator
    elseif node_type == TIMES
        left = @addr(covariance_prior(), :left)
        right = @addr(covariance_prior(), :right)
        node = Times(left, right)

    # unknown node type
    else
        error("Unknown node type: $node_type")
    end

    return node
end

@gen function model(xs::Vector{Float64})
    n = length(xs)
    covariance_fn::Node = @addr(covariance_prior(), :tree)
    noise = @addr(gamma(1, 1), :noise) + 0.01
    cov_matrix = compute_cov_matrix_vectorized(covariance_fn, noise, xs)
    @addr(mvnormal(zeros(n), cov_matrix), :ys)
    return covariance_fn
end;

In [ ]:
function serialize_trace(tr, xmin, xmax)
    choices = get_assmt(tr)
    (xs,) = get_args(tr)
    curveXs = collect(Float64, range(xmin, length=100, stop=xmax))
    curveYs = [predict_ys(get_retval(tr), 0.0001, xs, choices[:ys],curveXs) for i=1:5]
    Dict("y-coords" => choices[:ys],
        "curveXs" => curveXs, "curveYs" => curveYs)
end

In [ ]:
viz = Viz(server, "gp-viz/dist", [collect(Float64, -1:0.1:1)]);
for iter=1:20
    (tr, _) = initialize(model, (collect(Float64, -1:0.1:1),))
    putTrace!(viz, "t$(iter)", serialize_trace(tr, -1, 1))
end
displayInNotebook(viz)

In [ ]:
function initialize_trace(xs::Vector{Float64}, ys::Vector{Float64})
    constraints = DynamicAssignment()
    constraints[:ys] = ys
    (trace, _) = initialize(model, (xs,), constraints)
    return trace
end

In [ ]:
function dsl_expression(assmt::Assignment)
    node_type = assmt[:type]
    if node_type == CONSTANT
        return "$(assmt[:param])"
    elseif node_type == LINEAR
        return "LINEAR($(assmt[:param]))"
    elseif node_type == SQUARED_EXP
        return "SQUARED_EXP($(assmt[:length_scale]))"
    elseif node_type == PERIODIC
        return "PERIODIC(scale = $(assmt[:scale]), period = $(assmt[:period]))"
    elseif node_type == PLUS
        return "($(dsl_expression(get_subassmt(assmt, :left))) + $(dsl_expression(get_subassmt(assmt, :right))))"
    else
        return "($(dsl_expression(get_subassmt(assmt, :left))) * $(dsl_expression(get_subassmt(assmt, :right))))"
    end
end

In [ ]:
@gen function random_node_path(n::Node)
    if @addr(bernoulli(isa(n, LeafNode) ? 1.0 : 0.5), :stop)
        return :tree
    else
        (next_node, direction) = @addr(bernoulli(0.5), :left) ? (n.left, :left) : (n.right, :right)
        rest_of_path = @addr(random_node_path(next_node), :rest_of_path)
        if isa(rest_of_path, Pair)
            return :tree => direction => rest_of_path[2]
        else
            return :tree => direction
        end
    end
end

@gen function regen_random_subtree(prev_trace)
    @addr(covariance_prior(), :new_subtree)
    @addr(random_node_path(get_retval(prev_trace)), :path)
end

function subtree_involution(trace, fwd_assmt::Assignment, path_to_subtree, proposal_args::Tuple)
    # Need to return a new trace, a bwd_assmt, and a weight.
    model_assmt = get_assmt(trace)
    bwd_assmt = DynamicAssignment()
            
    set_subassmt!(bwd_assmt, :path, get_subassmt(fwd_assmt, :path))
    set_subassmt!(bwd_assmt, :new_subtree, get_subassmt(model_assmt, :tree))
    
    new_trace_update = DynamicAssignment()
    set_subassmt!(new_trace_update, path_to_subtree, get_subassmt(fwd_assmt, :new_subtree))
    (new_trace, weight, _, _) = force_update(get_args(trace), noargdiff, trace, new_trace_update)
    (new_trace, bwd_assmt, weight)
end


function run_mcmc(trace, iters::Int)
    for iter=1:iters
        (trace, _) = mh(trace, regen_random_subtree, (), subtree_involution)
        (trace, _) = mh(trace, select(:noise))
    end
    return trace
end

function extract_cov_noise(trace)
    cov = get_retval(trace)
    noise = get_assmt(trace)[:noise]
    return (cov, noise)
end

In [ ]:
#########################
# load airline data set #
#########################
import CSV
using StatsBase: mean
function get_airline_dataset()
    df = CSV.read("$(@__DIR__)/airline.csv")
    xs = df[1]
    ys = df[2]
    xs .-= minimum(xs) # set x minimum to 0.
    xs /= maximum(xs) # scale x so that maximum is at 1.
    ys .-= mean(ys) # set y mean to 0.
    ys *= 4 / (maximum(ys) - minimum(ys)) # make it fit in the window [-2, 2]
    return (xs, ys)
end

In [ ]:
(xs, ys) = get_airline_dataset();

In [ ]:
function run_mcmc(trace, iters::Int, viz)
    for iter=1:iters
        (trace, acc) = mh(trace, regen_random_subtree, (), subtree_involution)
        (trace, _) = mh(trace, select(:noise))
        if acc
            putTrace!(viz, "t", serialize_trace(trace, -1, 2))
            sleep(0.1)
        end
    end
    return trace
end

In [ ]:
viz = Viz(server, "gp-viz/dist", [xs])
t = initialize_trace(xs, ys);
putTrace!(viz, "t", serialize_trace(t, -1, 2))
t = displayInNotebook(viz) do
    sleep(1)
    run_mcmc(t, 1000, viz)
end;